In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_teddynote import logging
logging.langsmith('test_node')

# 제네시스 관련 질문 판단 노드 함수

In [ ]:
from RAG.types import State
from langgraph.graph import START, END
from RAG.llm.prompt_templates import get_genesis_manual_classification_prompt
from RAG.llm.model import get_OpenAI

In [ ]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
        """
        You are an expert in Genesis Q&A.  
        Determine whether the user's question is related to 'Genesis' or general vehicle usage and maintenance.  
        
        Criteria:  
        - If the question mentions 'Genesis' vehicles or models (e.g., G70, G80, G90) and asks about usage, maintenance, functionality, update methods, or other manual-related inquiries, answer "yes."  
        - If the question does not explicitly mention 'Genesis' but refers to general vehicle usage, maintenance, or functionality that could apply to any vehicle, including 'Genesis,' answer "yes."  
        - If the question is unrelated to vehicles or does not fit the above criteria, answer "no."  
        
        Assume that general vehicle-related questions (e.g., how to start a car, how to check oil) are indirectly relevant to 'Genesis' unless explicitly stated otherwise.  
        
        Your answer must be in English, and it should be either "yes" or "no."  
        
        #Question:  
        {question}  
        
        #Answer:


        """
    )

In [ ]:
# prompt = get_genesis_manual_classification_prompt()
llm = get_OpenAI()


In [ ]:
from langchain_core.output_parsers import StrOutputParser


chain = prompt | llm | StrOutputParser()
response = chain.invoke({"question":'시동거는 법?'})

print(response)

# 벡터DB 조회 노드 함수

In [ ]:
from RAG.types import State
from langgraph.graph import START, END
from RAG.llm.prompt_templates import get_genesis_manual_classification_prompt
from RAG.llm.model import get_OpenAI

In [ ]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
        """
        너는 전문적인 사용자 도우미이며, 주어진 CONTEXT를 기반으로 사용자의 질문에 대한 정확하고 상세한 답변을 작성해야 한다.  
        답변은 한글로 작성하며, 마크다운 형식을 사용해 가독성을 높여야 한다.  
        답변에는 항상 정보의 출처를 명시하며, CONTEXT를 통해 제공된 정보를 우선적으로 활용해야 한다.  
        필요한 경우, 추가적인 예시나 세부 정보를 포함해 사용자의 질문에 완벽하게 답변하라.  
        
        # Context:  
        {context}
        
        # Question:  
        {question}  
        
        # Answer:  
        ### 질문에 대한 답변:  
        질문에 대한 구체적인 답변 작성
        
        ### 출처:  
        1. CONTEXT 기반 정보: CONTEXT에서 관련된 정보 요약  

        """
    )

In [ ]:
llm = get_OpenAI()

In [ ]:
from RAG.database.milvus_connector import connect_to_milvus, get_collection
connect_to_milvus()
collection = get_collection('manual')

In [ ]:
from milvus_model.hybrid import BGEM3EmbeddingFunction

ef = BGEM3EmbeddingFunction(use_fp16=False, device="cpu")

In [ ]:
query_embeddings = ef(['시동 켜는법?'])

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
# !pip install langchain_milvus

In [ ]:
from pymilvus import (
    AnnSearchRequest,
    WeightedRanker,
)


def dense_search(col, query_dense_embedding, limit=10):
    search_params = {"metric_type": "IP", "params": {}}
    res = col.search(
        [query_dense_embedding],
        anns_field="dense_vector",
        limit=limit,
        output_fields=["text"],
        param=search_params,
    )[0]
    return [hit.get("text") for hit in res]


def sparse_search(col, query_sparse_embedding, limit=10):
    search_params = {
        "metric_type": "IP",
        "params": {},
    }
    res = col.search(
        [query_sparse_embedding],
        anns_field="sparse_vector",
        limit=limit,
        output_fields=["text"],
        param=search_params,
    )[0]
    return [hit.get("text") for hit in res]


def hybrid_search(
    col,
    query_dense_embedding,
    query_sparse_embedding,
    sparse_weight=1.0,
    dense_weight=1.0,
    limit=10,
):
    dense_search_params = {"metric_type": "IP", "params": {}}
    dense_req = AnnSearchRequest(
        [query_dense_embedding], "dense_vector", dense_search_params, limit=limit
    )
    sparse_search_params = {"metric_type": "IP", "params": {}}
    sparse_req = AnnSearchRequest(
        [query_sparse_embedding], "sparse_vector", sparse_search_params, limit=limit
    )
    rerank = WeightedRanker(sparse_weight, dense_weight)
    res = col.hybrid_search(
        [sparse_req, dense_req], rerank=rerank, limit=limit, output_fields=["text"]
    )[0]
    return [hit.get("text") for hit in res]

In [ ]:
dense_results = dense_search(collection, query_embeddings["dense"][0])
sparse_results = sparse_search(collection, query_embeddings["sparse"]._getrow(0))
hybrid_results = hybrid_search(
    collection,
    query_embeddings["dense"][0],
    query_embeddings["sparse"]._getrow(0),
    sparse_weight=0.7,
    dense_weight=1.0,
)


In [ ]:
retriever=vector_store_loaded.as_retriever(search_kwargs={"expr": 'namespace == "ankush"'})

In [ ]:
retriever.invoke('시동')

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = {"context": retriever, "question": RunnablePassthrough()}|prompt | llm | StrOutputParser()
response = chain.invoke({"question":'시동거는 법?'})



In [ ]:
from RAG.llm.prompt_templates import get_genesis_manual_classification_prompt, get_answer_with_context_prompt
from RAG.llm.model import get_OpenAI

from langchain_core.output_parsers import StrOutputParser

In [ ]:

from RAG.tools.tools import search_milvus

In [ ]:
from RAG.types import State
from langgraph.graph import START, END

from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [ ]:
query = '시동켜는법?'
context = search_milvus(query)
prompt = get_answer_with_context_prompt()
llm = get_OpenAI()



In [ ]:
prompt.invoke({'context': context, 'question': '시동'})

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
        """
        너는 전문적인 사용자 도우미이며, 주어진 CONTEXT만을 가지고 사용자의 질문에 대한 정확하고 상세한 답변을 작성해야 한다.  
        답변은 한글로 작성하며, 마크다운 형식을 사용해 가독성을 높여야 한다.  
        답변에는 항상 정보의 출처를 명시하며, CONTEXT를 통해 제공된 정보를 우선적으로 활용해야 한다.  
        필요한 경우, 추가적인 예시나 세부 정보를 포함해 사용자의 질문에 완벽하게 답변하라.  
        
        # Context:  
        {context}
        
        # Question:  
        {question}  
        
        # Answer:  
        ### 질문에 대한 답변:  
        질문에 대한 구체적인 답변 작성
        
        ### 출처:  
        1. CONTEXT 기반 정보: CONTEXT에서 관련된 정보 요약  

        """
    )


# context_list = '\n'.join(context)
# 
# pp = ({'context': context_list, 'question': RunnablePassthrough()} | prompt)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

pt = {"context": context, "question": RunnablePassthrough()} | prompt

# 데이터를 실행
output = pt.invoke({"question": "tlehd"})
print(output)

In [ ]:

pt =  prompt

# 실행
output = pt.invoke({"context": context, "question": '시동'})

In [ ]:
chain = prompt | llm | StrOutputParser()
output = chain.invoke({"context": context, "question": '경고등이 울리는데 왜 이래?'})


In [ ]:
print(output)

# 답변 점수 측정 노드

In [ ]:
from RAG.llm.prompt_templates import get_genesis_manual_classification_prompt, get_answer_with_context_prompt
from RAG.llm.model import get_OpenAI
from langchain_core.output_parsers import StrOutputParser

In [ ]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
        """
        너는 전문적인 사용자 도우미이며, 주어진 Question과 Answer를 가지고 Answer가 사용자가 원하는 답인지 점수를 계산해야 한다.  
        점수를 계산할 때, Answer가 주어진 Context외의 정보를 가지고 있으면 안된다.
        JSON 형식 Reason과 Score로 대답해라.
        # Context:  
        {context}
        
        # Question:  
        {question}  
        
        # Answer:  
        {answer}
        
        {{
          "reason": "value1",
          "score": 100
        }}     
        """
    )

In [ ]:
prompt.input_variables

In [ ]:
llm = get_OpenAI()

In [ ]:

from RAG.tools.tools import search_milvus
from RAG.database.milvus_connector import connect_to_milvus, get_collection
connect_to_milvus()
collection = get_collection('manual')
query = '시동켜는법?'
context = search_milvus(query)

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

# 사용자의 질문에 대한 답변
response_schemas = [
    ResponseSchema(
        name="score",
        description="질문에 대한 점수, 숫자여햐 한다.",
    ),
    ResponseSchema(name="reason", description="점수를 제외한 나머지 "),
]
# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
answer = """

    ### 질문에 대한 답변:  
    경고등이 울리는 이유는 여러 가지가 있을 수 있습니다. 일반적으로 경고등은 차량의 시스템에서 문제가 발생했음을 알리는 신호입니다. 다음은 경고등이 울릴 수 있는 몇 가지 일반적인 원인입니다:
    
    1. **브레이크 시스템 문제**: 브레이크 페달을 밟지 않고 시동 버튼을 누르면 시동이 걸리지 않거나 경고등이 켜질 수 있습니다. 브레이크 시스템에 문제가 있을 경우 경고등이 울릴 수 있습니다.
    
    2. **기어 상태**: 기어가 'P'(주차) 또는 'N'(중립) 상태가 아닐 경우, 시동이 걸리지 않거나 경고음이 발생할 수 있습니다. 차량이 주행 중일 때 기어를 잘못 조작하면 경고등이 울릴 수 있습니다.
    
    3. **스마트 키 문제**: 스마트 키의 배터리가 방전되었거나 차량 내부에 스마트 키가 없을 경우 경고등이 울릴 수 있습니다. 이 경우, 스마트 키가 차량에 있는지 확인해야 합니다.
    
    4. **엔진 상태**: 엔진이 정지하기 직전에는 고속 공회전을 삼가야 하며, 이로 인해 경고등이 울릴 수 있습니다. 엔진이 비정상적으로 작동할 경우 경고등이 켜질 수 있습니다.
    
    5. **전자식 파킹 브레이크**: 전자식 파킹 브레이크가 제대로 작동하지 않으면 경고등이 울릴 수 있습니다. 이 경우, 파킹 브레이크 스위치를 확인해야 합니다.
    
    경고등이 울릴 경우, 차량의 매뉴얼을 참조하거나 가까운 정비소에 문의하여 정확한 원인을 파악하고 조치를 취하는 것이 중요합니다.
    
    ### 출처:  
    1. CONTEXT 기반 정보: 차량의 시동 및 주행 관련 정보, 경고등 및 시스템 문제에 대한 설명.
"""

In [ ]:
# query = '시동켜는법?'
query = '경고등이 울리는데 왜 이래?'
context = search_milvus(query)

res= chain.invoke({"context": context, "question": query, "answer": answer})

In [ ]:
print(res)

In [ ]:
res['score']

In [ ]:
def rewrite_query(query, llm):
    prompt = PromptTemplate(
        input_variables=["query"],
        template="Rewrite the query '{query}' to improve retrieval quality."
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain.run(query)

In [ ]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
        """
        Rewrite the query '{question}' to improve retrieval quality. Use the context of previous questions and generate only one improved query. Ensure the new query does not overlap with any of the previous questions.

        # Previous questions: 
        {previous}

        """
    )

In [ ]:
llm = get_OpenAI()

In [ ]:
chain = prompt | llm | StrOutputParser()

In [ ]:
query = '시동 켜는 법'
previous = ['자동차 시동 켜는 방법','시동을 켜는 절차']
res = chain.invoke({'previous':previous,'question': query})
res

In [5]:
from RAG.types import State 
a = State({"message":"dfdf"})

In [6]:
a['context']

KeyError: 'context'